In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
# from pprint import pprint
# from lxml import html
# from lxml.html import fromstring
# import urllib.request
from urllib.request import urlopen
# import random
import re
# import scrapy

## BeautifulSoup

In [3]:
# Preparación BeautifulSoup
url = 'https://racingnews365.com/formula-1-driver-salaries-2021'
html = requests.get(url)
soup = BeautifulSoup(html.content, 'html.parser')

## Webscraping

In [4]:
# Selección de tabla dentro de la web racingnews
table = soup.find('table',{'class':"table-default table-default--expanded content-field__table"}) 

# Selección de filas dentro de la tabla y asignarlas a variable rows
rows = [] 
for table in table.find_all('tbody'):
    table = table
    for row in table.find_all('tr'):
        rows.append(row.getText())
        
# Vista de resultado        
rows[0:3]

['\n\n                            1\n\t\t\t\t\t\t\n\n                            Lewis Hamilton\n\t\t\t\t\t\t\n\n                            Mercedes\n\t\t\t\t\t\t\n\n                            $30m\n\t\t\t\t\t\t\n\n                            2021\n\t\t\t\t\t\t\n',
 '\n\n                            2\n\t\t\t\t\t\t\n\n                            Max Verstappen\n\t\t\t\t\t\t\n\n                            Red Bull\n\t\t\t\t\t\t\n\n                            $25m\n\t\t\t\t\t\t\n\n                            2023\n\t\t\t\t\t\t\n',
 '\n\n                            3\n\t\t\t\t\t\t\n\n                            Fernando Alonso\n\t\t\t\t\t\t\n\n                            Alpine\n\t\t\t\t\t\t\n\n                            $20m\n\t\t\t\t\t\t\n\n                            2022\n\t\t\t\t\t\t\n']

## Cleaning

In [5]:
cleaned_rows = [] # Lista en la que incluiremos las filas limpias.

def clean_rows(x): # Función para limpiar el texto dentro de cada fila
    for i in rows:
        i = i.replace('\n\n                            ','')
        i = i.replace('\n\t\t\t\t\t\t',',')
        i = i.replace(',\n','')
        cleaned_rows.append(i)  
        
clean_rows(rows) # Aplicamos función de limpieza a nuesta lista rows
cleaned_rows[0:3]

['1,Lewis Hamilton,Mercedes,$30m,2021',
 '2,Max Verstappen,Red Bull,$25m,2023',
 '3,Fernando Alonso,Alpine,$20m,2022']

## To pandas

In [6]:
# Dataframe con cada fila ya limpia, y separando columnas por comas con función split
df_drivers = pd.DataFrame([sub.split(",") for sub in cleaned_rows]) 

# Asignamos los nombres de cada columna al dataframe
df_drivers.columns = ['Number', 'Driver', 'Team', 'Salary', 'Expiry date']

# Asignamos columna number como index y eliminamos el que había
df_drivers.Number = range(1,21)
df_drivers.set_index('Number', inplace = True)

df_drivers

,Driver,Team,Salary,Expiry date
Number,,,,
1,Lewis Hamilton,Mercedes,$30m,2021
2,Max Verstappen,Red Bull,$25m,2023
3,Fernando Alonso,Alpine,$20m,2022
4,Sebastian Vettel,Aston Martin,$15m,2021*
5,Daniel Ricciardo,McLaren,$15m,2022*
6,Charles Leclerc,Ferrari,$12m,2024
7,Valtteri Bottas,Mercedes,$10m,2021
8,Carlos Sainz,Ferrari,$10m,2022*
9,Lance Stroll,Aston Martin,$10m,Open


In [8]:
# Cambio el formato de los salarios

Salary2 = []
for s in df_drivers.Salary:
    s = s.replace('m','000000')
    s = s.replace('k', '000')
    s = s.replace('$','')
    s = int(s)
    Salary2.append(s)

Salary_2 = Salary2
df_drivers['Salary_2'] = Salary_2

# Elimino asterisco final de 'Expiry date' con funcion map y una función lambda.
df_drivers['Expiry date'] = df_drivers['Expiry date'].map(lambda x: x.replace('*',''))

# Guardo df_drivers como csv en mi carpeta del proyecto
df_drivers.to_csv('../Hey-I-am-a-data-analyst/archive/archivef1/driverSalaries.csv' )
